In [ ]:
#!import "../0-setup/Packages.ipynb"

## Replicator

A developer can create any number of replicators.
They give them a name and an environment (typically "dev", "test", or "prod").
Both name and environment are mutable properties.

In [ ]:
[FactType("RaaS.Replicator")]
public record Replicator(User user, Guid identifier);

[FactType("RaaS.Replicator.Name")]
public record ReplicatorName(Replicator replicator, string value, ReplicatorName[] prior);

[FactType("RaaS.Replicator.Environment")]
public record ReplicatorEnvironment(Replicator replicator, string value, ReplicatorEnvironment[] prior);

[FactType("RaaS.Replicator.Deleted")]
public record ReplicatorDeleted(Replicator replicator, DateTime deletedAt);

Renderer.RenderTypes(typeof(Replicator), typeof(ReplicatorName), typeof(ReplicatorEnvironment), typeof(ReplicatorDeleted))

## Authentication

A developer can configure authentication mechanisms for replicators.
Supported authentication mechanisms include Apple and Google.
Given the requirements of each of those providers, these authentication mechanisms take different parameters.

While the end user authenticates with Apple or Google, they use that identity to further authenticate with the replicator.
This phase is a standard OAuth2 authorization code flow.
The developer supplies callback URLs for this flow.

All of this configuration information is shared with an environment.
This environment is well known to the developer.

In [ ]:
[FactType("RaaS.Environment")]
public record RaaSEnvironment(User creator, string environmentId);

[FactType("RaaS.Replicator.AuthenticationServiceRequest")]
public record AuthenticationServiceRequest(
    RaaSEnvironment environment,
    Replicator replicator,
    string callbackUrls,
    AuthenticationServiceRequest[] prior);

[FactType("RaaS.Replicator.AuthenticationApple")]
public record AuthenticationApple(
    RaaSEnvironment environment, 
    Replicator replicator, 
    string? appDescription, 
    string appBundleId, 
    string? serviceDescription, 
    string serviceIdentifier, 
    string keyName, 
    string teamId, 
    string keyId, 
    string keySecret, 
    AuthenticationApple[] prior);

[FactType("RaaS.Replicator.AuthenticationGoogle")]
public record AuthenticationGoogle(
    RaaSEnvironment environment,
    Replicator replicator,
    string clientId,
    string clientSecret,
    AuthenticationGoogle[] prior);

Renderer.RenderTypes(typeof(RaaSEnvironment), typeof(AuthenticationServiceRequest), typeof(AuthenticationApple), typeof(AuthenticationGoogle))

## RaaS Portal Administrators

Administrators represent people who can configure an environment.
They grant privileges to other administrators and to service principals.
That permission can be revoked.

The creator of the environment grants permission to the first administrator.
Thereafter, an administrator can add new administrators and revoke privileges.

In [ ]:
[FactType("RaaS.Administrator")]
public record RaaSAdministrator(User user, RaaSEnvironment environment, DateTime createdAt);

[FactType("RaaS.Administrator.Revoke")]
public record RaaSAdministratorRevoke(RaaSAdministrator administrator);

Renderer.RenderTypes(typeof(RaaSAdministrator), typeof(RaaSAdministratorRevoke))

## RaaS Portal Service Principals

Service principals represent machines that perform functions within a RaaS portal environment.
Those functions include configuring authentication providers and generating secrets.

A portal administrator creates a service principal.
Service principals cannot grant permissions to others.

In [ ]:
[FactType("RaaS.ServicePrincipal")]
public record RaaSServicePrincipal(User user, RaaSEnvironment environment, DateTime createdAt);

[FactType("RaaS.ServicePrincipal.Revoke")]
public record RevokeRaaSServicePrincipal(RaaSServicePrincipal servicePrincipal);

Renderer.RenderTypes(typeof(RaaSServicePrincipal), typeof(RevokeRaaSServicePrincipal))

## Request Completions

A portal service principal processes the authentication and endpoint requests.
It responds with a completion fact that sometimes carries generated information.

For example, upon completing a request for an endpoint, the host provides a URL.
And when completing a request for an OAuth2 authentication service, the host responds with the public key that will be used to validate JWT tokens.

In [ ]:
[FactType("RaaS.RequestForEndpoint")]
public record RequestForEndpoint(RaaSEnvironment environment, Replicator replicator, DateTime createdAt);

[FactType("RaaS.Endpoint")]
public record Endpoint(RequestForEndpoint request, string url);

[FactType("RaaS.CompletedAuthenticationServiceRequest")]
public record CompletedAuthenticationServiceRequest(AuthenticationServiceRequest request, string publicKey);

[FactType("RaaS.CompletedAuthenticationApple")]
public record CompletedAuthenticationApple(AuthenticationApple authenticationApple);

[FactType("RaaS.CompletedAuthenticationGoogle")]
public record CompletedAuthenticationGoogle(AuthenticationGoogle authenticationGoogle);

Renderer.RenderTypes(typeof(RequestForEndpoint), typeof(Endpoint), typeof(CompletedAuthenticationServiceRequest), typeof(CompletedAuthenticationApple), typeof(CompletedAuthenticationGoogle))

## Secrets

When setting up authorization and distribution rules, the developer must provide a shared secret.
They request a secret from the environment.
A service principal responds with a randomly generated secret that can be used to authenticate a CI/CD server.

In [ ]:
[FactType("RaaS.Replicator.SecretRequest")]
public record ReplicatorSecretRequest(RaaSEnvironment environment, Replicator replicator, ReplicatorSecretRequest[] prior);

[FactType("RaaS.Replicator.Secret")]
public record ReplicatorSecret(ReplicatorSecretRequest request, string secret);

Renderer.RenderTypes(typeof(ReplicatorSecretRequest), typeof(ReplicatorSecret))